Author: Ronny F. Pacheco Date: Oct 2024
Copyright: © 2024 Ronny Pacheco License: MIT License

---

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.    

# Needed modules

In [13]:
# Load the needed libraries
import pickle
import os
import pandas as pd

In [2]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore


@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [3]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    notebook_name = os.path.basename(os.path.abspath(''))
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            dict_variables = {
            }
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as pickle_file:
            variables = pickle.load(pickle_file)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            print(f"* Loading variable: {pickle_key}")
            globals()[pickle_key] = pickle_value
# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="load")

Skipping cell


# Prepare data

In [4]:
lre_df = pd.read_csv('./data_1.1/lre_to_filter.csv', sep=',', header=0)
print(lre_df.shape)
print(lre_df.dtypes)
lre_df.head()

(881, 16)
neg_name         object
chrom            object
start             int64
end               int64
strand           object
gene_id          object
transcript_id    object
parent_id        object
gene               bool
transcript         bool
CDS                bool
3utr               bool
5utr               bool
pseudogen        object
notes            object
interval         object
dtype: object


,neg_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval
0,lre_2.1,LinJ.02,93941,102762,-,LINF_020007000,NaN,NaN,True,False,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]"
1,lre_2.1,LinJ.02,93941,102762,-,NaN,LINF_02T0007000,LINF_020007000,False,True,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]"
2,lre_2.1,LinJ.02,93941,99642,-,NaN,NaN,LINF_02T0007000,False,False,False,True,False,NaN,NaN,"[93941, 99642]"
3,lre_2.2,LinJ.02,137296,139716,-,LINF_020007950,NaN,NaN,True,False,False,False,False,NaN,NaN,"[137296, 139716]"
4,lre_2.2,LinJ.02,137296,139716,-,NaN,LINF_02T0007950,LINF_020007950,False,True,False,False,False,NaN,NaN,"[137296, 139716]"


In [5]:
# Change name from 'neg_name' to 'lre_name'
lre_df.rename(columns={'neg_name': 'lre_name'}, inplace=True)

In [6]:
# Let's add a new column about the "neg_family"
# let's take the number "x" from lre_x.1, where 'x' is the number to add to a new column

# Define a function to extract the number x from neg_name column
def extract_neg_group(column_name):
    import re
    match = re.search(r'lre_(\d+)\.\d+', column_name)
    return int(match.group(1)) if match else None

# Apply the function to create a new column
lre_df['lre_family'] = lre_df['lre_name'].apply(extract_neg_group)

# Display the updated dataframe
lre_df.head()

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
0,lre_2.1,LinJ.02,93941,102762,-,LINF_020007000,NaN,NaN,True,False,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]",2
1,lre_2.1,LinJ.02,93941,102762,-,NaN,LINF_02T0007000,LINF_020007000,False,True,False,False,False,NaN,phosphoglycan_beta_1-3_galactosyltransferase,"[93941, 102762]",2
2,lre_2.1,LinJ.02,93941,99642,-,NaN,NaN,LINF_02T0007000,False,False,False,True,False,NaN,NaN,"[93941, 99642]",2
3,lre_2.2,LinJ.02,137296,139716,-,LINF_020007950,NaN,NaN,True,False,False,False,False,NaN,NaN,"[137296, 139716]",2
4,lre_2.2,LinJ.02,137296,139716,-,NaN,LINF_02T0007950,LINF_020007950,False,True,False,False,False,NaN,NaN,"[137296, 139716]",2


# Load functions

In [27]:
def search_string(data_frame, searching_string):
    """
    :param data_frame: The DataFrame containing the data to be searched.
    :param searching_string: The string to search for within the 'notes' column of the DataFrame.
    :return: A filtered DataFrame containing rows where 'notes' contain the search string (case-insensitive).
    """
    filtered_df = data_frame[data_frame['notes'].fillna('').str.contains(searching_string, case=False)]
    print(f"The of filtered data: {filtered_df.shape}")
    print(f"The unique values in column 'lre_name': {filtered_df['lre_name'].nunique()}")
    return filtered_df

def filter_and_display(data_frame, search_term):
    """
    :param data_frame: The DataFrame containing the data to be filtered.
    :param search_term: The term to search for within the DataFrame.
    :return: A filtered DataFrame with specific columns where the "gene" column is True.
    """
    filtered_data = search_string(lre_df, search_term)
    print("=" * 50)
    print(filtered_data['notes'].value_counts())
    print("=" * 50)
    filtered_result = filtered_data.loc[
        filtered_data["gene"] == True,
        ['lre_name', 'lre_family', 'chrom', 'gene_id', 'notes', 'start', 'end']
    ]
    return filtered_result

def checking_data(data_frame):
    """
    :param data_frame: A pandas DataFrame that contains the dataset to be checked.
    :return: None. The function prints the shape of the DataFrame and the number of unique values in the 'lre_name' column.
    """
    print(f"Shape of the data frame is: {data_frame.shape}")
    print(f"Number of unique values in column 'lre_name': {data_frame['lre_name'].nunique()} ")

def check_family(data_frame, family_name):
    """
    :param data_frame: The input pandas DataFrame containing genomic data with columns such as 'lre_name', 'chrom', 'gene_id', 'notes', 'start', and 'end'.
    :param family_name: The family name pattern (string) to be searched at the end of 'lre_name'.
    :return: A DataFrame filtered to include rows where 'lre_name' contains the given family name pattern at the end and 'gene' is True. The returned DataFrame includes columns 'lre_name', 'chrom', 'gene_id', 'notes', 'start', and 'end'.
    """
    pattern = fr'\d+{family_name}$'
    filtered_data = data_frame.loc[
        (data_frame['lre_name'].str.contains(pattern, na=False)) & (data_frame['gene'] == True),
        ['lre_name', 'chrom', 'gene_id', 'notes', 'start', 'end']
    ]
    return filtered_data
 

def group_and_count(data_frame, group_column='lre_name'):
    """
    :param data_frame: The pandas DataFrame containing the data to be processed.
    :param group_column: The column name on which to group the DataFrame.
    :return: A DataFrame containing the count of unique 'notes' values, sorted in descending order.
    """
    grouped_df = data_frame.groupby(group_column)
    grouped_column_counter = {}
    loc_counter_global = {}
    for _, group_data in grouped_df:
        notes = group_data['notes'].unique()
        for element in notes:
            if element is not None:
                if element not in grouped_column_counter:
                    grouped_column_counter[element] = 1
                else:
                    grouped_column_counter[element] += 1
                    
        loc_data = group_data[["gene", "transcript", "CDS", "3utr", "5utr"]].sum()
        for loc_element in loc_data.index:
            if loc_data[loc_element] > 0:
                if loc_element not in loc_counter_global:
                    loc_counter_global[loc_element] = 1
                else:
                    loc_counter_global[loc_element] += 1
        
    notes_counter_global_sorted = dict(sorted(grouped_column_counter.items(), key=lambda x: x[1], reverse=True))
    notes_counter_global_sorted_df = pd.DataFrame(notes_counter_global_sorted.items(), columns=["notes", "count"])
    print(loc_counter_global)
    return notes_counter_global_sorted_df

def accept_group(data_frame, filtered_data, good_df, keep_filtering_df):
    """
    :param data_frame: The main DataFrame that contains data to be processed.
    :param filtered_data: A DataFrame containing the subset of `data_frame` that meets certain criteria.
    :param good_df: A DataFrame that accumulates 'good' data from `data_frame`.
    :param keep_filtering_df: A DataFrame that requires further filtering.
    :return: A tuple containing `good_df` and `keep_filtering_df`. 
    """
    print('='*50)
    print('Data to filter:')
    all_data = data_frame[data_frame['lre_name'].isin(filtered_data['lre_name'])]
    checking_data(all_data)
    
    if good_df:
        print('='*50)
        print('Good lre elements:')
        good_df = pd.concat([good_df, all_data])
        checking_data(good_df)
    else:  # If it doesn't exist
        good_df = all_data
        print('='*50)
        print('Good lre elements:')
        checking_data(good_df)
    
    print('='*50)
    print('Keep filtering df:')
    keep_filtering_df = keep_filtering_df[~keep_filtering_df['lre_name'].isin(filtered_data['lre_name'])]
    checking_data(keep_filtering_df)
        
    return good_df, keep_filtering_df

# Filtering

## Descriptive stats

In [8]:
checking_data(lre_df)
group_and_count(lre_df, 'lre_name')

Shape of the data frame is: (881, 17)
Number of unique values in column 'lre_name': 307 
{'gene': 307, 'transcript': 307, '3utr': 184, '5utr': 20, 'CDS': 2}


,notes,count
0,NaN,307
1,hypothetical_protein_-_conserved,32
2,amastin_surface_glycoprotein_-_putative,23
3,protein_of_unknown_function_-_conserved,8
4,glucose_transporter,8
...,...,...
89,tRNA-Leu,1
90,engulfment_and_cell_motility_domain_2_-_putative,1
91,Protein_kinase_domain-containing_protein,1
92,protein-l-isoaspartate_o-methyltransferase_-_p...,1


## Filtering

### Hypothetical protein

In [31]:
filter_df = filter_and_display(lre_df, 'hypothetical_protein')
filter_df

The of filtered data: (74, 17)
The unique values in column 'lre_name': 37
notes
hypothetical_protein_-_conserved    64
hypothetical_protein                10
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
10,lre_3.3,3,LinJ.04,LINF_040010300,hypothetical_protein_-_conserved,208282,210473
25,lre_5.2,5,LinJ.05,LINF_050009400,hypothetical_protein_-_conserved,144533,148377
124,lre_19.2,19,LinJ.12,LINF_120012900,hypothetical_protein_-_conserved,393788,397393
130,lre_19.3,19,LinJ.12,LINF_120013300,hypothetical_protein_-_conserved,406344,409449
184,lre_24.2,24,LinJ.15,LINF_150014000,hypothetical_protein_-_conserved,306215,309145
202,lre_27.2,27,LinJ.17,LINF_170019100,hypothetical_protein_-_conserved,580021,585507
205,lre_26.2,26,LinJ.17,LINF_170022200,hypothetical_protein_-_conserved,687597,689967
210,lre_28.1,28,LinJ.19,LINF_190009700,hypothetical_protein_-_conserved,182279,187638
237,lre_32.1,32,LinJ.19,LINF_190019900,hypothetical_protein_-_conserved,615371,617193
260,lre_34.1,34,LinJ.23,LINF_230013900,hypothetical_protein,286043,296768


In [10]:
filter_df['lre_family'].value_counts()

lre_family
19    4
44    4
49    4
61    4
96    4
26    2
27    2
3     2
32    2
28    2
36    2
34    2
41    2
46    2
5     2
24    2
48    2
47    2
60    2
50    2
64    2
65    2
66    2
58    2
69    2
81    2
87    2
86    2
89    2
92    2
95    2
98    2
Name: count, dtype: int64

In [23]:
good_lre, reduced_lre = accept_group(lre_df, filter_df, None, lre_df)

Data to filter:
Shape of the data frame is: (121, 17)
Number of unique values in column 'lre_name': 37 
Good lre elements:
Shape of the data frame is: (121, 17)
Number of unique values in column 'lre_name': 37 
Keep filtering df:
Shape of the data frame is: (760, 17)
Number of unique values in column 'lre_name': 270 


In [24]:
group_and_count(reduced_lre)

{'gene': 270, 'transcript': 270, '3utr': 151, '5utr': 14, 'CDS': 2}


,notes,count
0,NaN,270
1,amastin_surface_glycoprotein_-_putative,23
2,protein_of_unknown_function_-_conserved,8
3,glucose_transporter,8
4,amastin-like_protein,7
...,...,...
85,tRNA-Leu,1
86,engulfment_and_cell_motility_domain_2_-_putative,1
87,Protein_kinase_domain-containing_protein,1
88,protein-l-isoaspartate_o-methyltransferase_-_p...,1


### Amastin

In [36]:
filter_df = filter_and_display(lre_df, 'amastin_surface_glycoprotein_-_putative')
filter_df

The of filtered data: (46, 17)
The unique values in column 'lre_name': 23
notes
amastin_surface_glycoprotein_-_putative    46
Name: count, dtype: int64


,lre_name,lre_family,chrom,gene_id,notes,start,end
605,lre_73.9,73,LinJ.34,LINF_340024200,amastin_surface_glycoprotein_-_putative,805687,808315
641,lre_77.1,77,LinJ.34,LINF_340034300,amastin_surface_glycoprotein_-_putative,1240773,1245780
643,lre_78.1,78,LinJ.34,LINF_340034300,amastin_surface_glycoprotein_-_putative,1240773,1245780
645,lre_79.1,79,LinJ.34,LINF_340034300,amastin_surface_glycoprotein_-_putative,1240773,1245780
650,lre_77.2,77,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
652,lre_78.2,78,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
654,lre_79.2,79,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
656,lre_80.1,80,LinJ.34,LINF_340034400,amastin_surface_glycoprotein_-_putative,1246328,1249925
664,lre_77.3,77,LinJ.34,LINF_340034500,amastin_surface_glycoprotein_-_putative,1252201,1255794
666,lre_78.3,78,LinJ.34,LINF_340034500,amastin_surface_glycoprotein_-_putative,1252201,1255794


In [37]:
filter_df['lre_family'].value_counts()

lre_family
77    6
79    6
78    6
80    4
73    1
Name: count, dtype: int64

In [40]:
lre_df[
    (lre_df['gene']==True) & (lre_df['lre_family'] == 79)
]

,lre_name,chrom,start,end,strand,gene_id,transcript_id,parent_id,gene,transcript,CDS,3utr,5utr,pseudogen,notes,interval,lre_family
645,lre_79.1,LinJ.34,1240773,1245780,+,LINF_340034300,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1240773, 1245780]",79
654,lre_79.2,LinJ.34,1246328,1249925,+,LINF_340034400,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1246328, 1249925]",79
668,lre_79.3,LinJ.34,1252201,1255794,+,LINF_340034500,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1252201, 1255794]",79
682,lre_79.4,LinJ.34,1258088,1261692,+,LINF_340034600,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1258088, 1261692]",79
696,lre_79.5,LinJ.34,1263977,1267600,+,LINF_340034700,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1263977, 1267600]",79
708,lre_79.6,LinJ.34,1269875,1273542,+,LINF_340034800,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein,"[1269875, 1273542]",79
718,lre_79.7,LinJ.34,1275817,1279464,+,LINF_340034900,NaN,NaN,True,False,False,False,False,NaN,amastin_surface_glycoprotein_-_putative,"[1275817, 1279464]",79
